In [ ]:
"""
The notebook consits of two parts, both in different cells.

Cell 1 : 

Author: Nick Schaub (nick.schaub@nih.gov)

Description: This cell contains WippPy which consists of classes for interacting with WIPP.


Cell 2 :

Authors: Nick Schaub (nick.schaub@nih.gov)
         Gauhar Bains (gauhar.bains@labshare.org)
        
Description: This consists of the main UI and logic for the Neuroglancer notebook.



"""





import json as json_lib
import requests, copy, re
from pathlib import Path
import logging

# Initialize the logger
logging.basicConfig(format='%(asctime)s - %(name)-8s - %(levelname)-8s - %(message)s',
                    datefmt='%d-%b-%y %H:%M:%S')
logger = logging.getLogger("wipp")
logger.setLevel(logging.WARNING)

class WippData(object):
    """ Wipp data superclass
    
    This class should be implemented by all Wipp data type classes.
    
    """
    _entry_point = None
    _data_type_name = None
    
    #set api route
    api_route = ''
    
    _headers = {'Content-Type': 'application/json'}
    _logger = logging.getLogger('wipp.Data')
    
    def __init__(self,name=None,create=False,json=None,api_route=None,**kwargs):
        if api_route != None:
            self._logger.info('api_route: {}'.format(api_route))
            self.api_route = api_route
        if create and name != None:
            self._logger.info('create(): {}'.format(name))
            kwargs['data'] = {'name': name}
            self.json = self.create(**kwargs)
        elif 'data' in kwargs:
            self._logger.info('create(): attempting to create instance of {}'.format(self.__class__.__name__))
            self.json = self.create(**kwargs)
        elif json:
            self._logger.debug('creating object using json')
            self.json = json
        else:
            self.json = self._get(**kwargs)
    
        if self.json!=None:
            for key,value in self.json.items():
                self._logger.debug('setattr(): {}={}')
                setattr(self,key,value)
        
    def __repr__(self):
        return f'{self.name} (id: {self.id})'
    
    def delete(self):
        self._logger.info('delete(): {}'.format(self.api_route + self._entry_point + '/' + self.id))
        requests.delete(self.api_route + self._entry_point + '/' + self.id)
        
    def create(self,**kwargs):
        self._logger.info('create(): {}'.format(self.api_route + self._entry_point))
        return self._post(**kwargs)
        
    def _post(self,**kwargs):
        self._logger.debug('_post(): endpoint={}'.format(self.api_route + self._entry_point))
        
        config = {key:value for key,value in kwargs.items() if key in ['params','headers']}
        if 'data' in kwargs:
            config['data'] = json_lib.dumps(kwargs['data'])
        if 'headers' not in config.keys():
            config['headers'] = self._headers
        
        for key,val in config.items():
            self._logger.debug('_post(): {}={}'.format(key,val))
        
        if 'entrypoint' not in kwargs.keys():
            entrypoint = self._entry_point
        else:
            entrypoint = kwargs['entrypoint']
            
        r = requests.post(self.api_route + entrypoint,**config)
        self._logger.debug('_post(): status_code={}'.format(r.status_code))
        if r.status_code==201 or r.status_code==200:
            return r.json()
        elif r.status_code==409:
            self._logger.warning('_post(): Plugin already exists.')
        else:
            self._logger.critical('_post(): message={}'.format(r.text),exc_info=True)
            raise ValueError(self.__class__.__name__ + ' Error (status code {}): {}'.format(r.status_code,r.text))
        
    def _get(self,entrypoint=None,**kwargs):
        if entrypoint==None:
            entrypoint=self._entry_point
        self._logger.debug('_get(): endpoint={}'.format(self.api_route + entrypoint))
        
        config = {key:value for key,value in kwargs.items() if key in ['params','headers','data']}
        if 'data' in kwargs:
            config['data'] = json_lib.dumps(kwargs['data'])
        if 'headers' not in config.keys():
            config['headers'] = self._headers
        
        for key,val in config.items():
            self._logger.debug('_get(): {}={}'.format(key,val))
        
        r = requests.get(self.api_route + entrypoint,**config)
        self._logger.debug('_get(): status_code={}'.format(r.status_code))
        if r.status_code==200:
            return r.json()
        else:
            self._logger.critical('_get(): message={}'.format(r.text))
            raise ValueError(self.__class__.__name__ + ' Error (status code {}): {}'.format(r.status_code,r.text))
            
    @classmethod
    def setWippUrl(cls,url):
        cls._logger.info('setWippUrl(): {}'.format(url))
        cls.api_route = url
            
    @classmethod
    def all(cls,entry_point=False):
        """Get all instances of a data type

        Args:
            cls: Class reference for handling a WIPP data type
            entry_point: API entry point, appended to api path

        Returns:
            A dictionary, where the keys are hashes referencing a data
            instance and values are data_class objects.
        """
        if not entry_point:
            entry_point = cls._entry_point
        cls._logger.info('all(): getting all instances...')
        page = 0
        numel = 1000
        r = requests.get(cls.api_route + entry_point,params={'page':page,'size':numel})
        if r.status_code==200:
            all_data = r.json()['_embedded'][cls._entry_point]
            data = {}
            for datum in all_data:
                data[datum['id']] = cls(json=datum)
                cls._logger.debug('all(): object={}'.format(data[datum['id']]))
            for i in range(r.json()['page']['totalPages']-1):
                page += 1
                r = requests.get(cls.api_route + entry_point,params={'page':page,'size':numel})
                if r.status_code==200:
                    all_data = r.json()['_embedded'][cls._entry_point]
                    data = {}
                    for datum in all_data:
                        data[datum['id']] = cls(json=datum)
                        cls._logger.debug('all(): object={}'.format(data[datum['id']]))
        else:
            data = {}
        return data
    
    @classmethod
    def get_by_id(cls,oid):
        """Get data by id

        Args:
            cls: Class reference for handling a WIPP data type
            oid: Hash reference of data to access

        Returns:
            An object of type cls
        """    
        cls._logger.debug('get_by_id(): oid={}'.format(oid))
        r = requests.get(cls.api_route + cls._entry_point + '/' + oid)
        if r.status_code==200:
            instance = cls(json=r.json())
        else:
            cls._logger.warning('get_by_id(): returning NoneType')
            instance = None
        return instance
    
    @staticmethod
    def get_name(dtype,value):
        """ Get the name of a data instance

        Args:
            dtype: WIPP data type
            value: Unique hash reference

        Returns:
            A string containing the human readable dataset name
        """
        for cls in WippData.__subclasses__():
            if dtype==cls._entry_point:
                cls._logger.debug('get_name(): finding object associated with id={}'.format(value))
                return cls.all()[value].name
        
class WippJob(WippData):
    """ Class to handle WIPP Jobs

    Attributes:
        name: the name given to the WIPP job
        id: a unique hash assigned to the WIPP job
        json: The raw json returned by the WIPP Job backend query
        status: execution status of the WIPP job
        plugin_id: a unique hash assigned to the plugin used by the WIPP job
        plugin_name: the name of the WIPP plugin executed by the job
        inputs: the plugin input keys and values for the job
        outputs: the plugin output keys and values for the job

    Class Methods:
        get_all(): Returns a dictionary of all jobs {job hash: WippJob object}
        get(jid): Returns job with hash equal to jid

    Object Methods:
        delete(): Delete the job from WIPP.
        create(): Create the job in WIPP.
    """
    _entry_point = 'jobs'
    _data_type_name = 'Job'
    _logger = logging.getLogger('wipp.Data.WippJob')

    # Job template
    _payload = {'name': '',            # name of job
                'wippExecutable': '',  # plugin id
                'type': '',            # name of the plugin
                'dependencies': [],    # job ids for dependencies
                'parameters': {},      # dictionary of parameters
                'outputParameters': {},# dictionary of output parameters
                'wippWorkflow': ''}    # wipp workflow id
        
    def __repr__(self):
        return f'{self.name} (id: {self.id})'
    
class WippWorkflow(WippData):
    """ Class to handle WIPP Workflows

    Attributes:
        name: the name given to the WIPP workflow
        id: a unique hash assigned to the WIPP workflow
        json: the raw json returned by the WIPP Workflow backend query
        status: the execution status of the workflow
        link: a url to the backend workflow json
    
    Class Methods:
        get_all(): Returns a dictionary of all workflows{workflow hash: WippWorkflow object}
        get(wid): Returns workflow with hash equal to wid
    
    Object Methods:
        delete(): Delete the workflow from WIPP.
        create(): Create the workflow in WIPP.
        jobs(): Returns dictionary of all jobs in workflow, {job hash: WippJob object}
    """
    _entry_point = 'workflows'
    _data_type_name = 'Workflow'
    _logger = logging.getLogger('wipp.Data.WippWorkflow')
    
    def jobs(self):
        self._logger.info('jobs(): Getting all jobs for workflow={}'.format(self.id))
        if self.id not in WippWorkflow.all().keys():
            self._logger.critical('jobs(): Could not find workflow jobs')
            raise KeyError('Invalid workflow id.')
        
        r = self._get(entrypoint='jobs/search/findByWippWorkflow?wippWorkflow='+self.id)
        
        jobs = {job_json['id']:WippJob(json=job_json) for job_json in r['_embedded']['jobs']}
        for job in jobs.values():
            self._logger.debug('jobs(): job='.format(job))
            
        return jobs
    
    def update(self):
        self._logger.info('update(): updating workflow - {}'.format(self.name))
        wf = WippWorkflow.get_by_id(self.id)
        for key,value in wf.json.items():
            setattr(self,key,value)
    
    def submit(self):
        self._post(entrypoint='workflows/' + self.id + '/submit',parameters={'wippWorkflow': self.id})
    
    def add_job(self,plugin_name,job_name,inputs,plugin_version=None):
        payload = copy.deepcopy(WippJob._payload)
        plugin = WippPlugin.get_by_name(plugin_name,plugin_version)
        dependency_pattern = r'\{\{ (.*)\.(.*) \}\}'
        self._logger.info('add_job(): job_name={}, plugin_name={}, plugin_version={}'.format(job_name,plugin_name,plugin_version))
        
        # Add basic info to the payload
        payload['name'] = job_name
        payload['wippExecutable'] = plugin.id
        payload['type'] = plugin.name
        payload['wippWorkflow'] = self.id
        
        # validate and set inputs
        for inp in plugin.inputs:
            if inp['name'] not in inputs.keys() and inp['required']:
                self._logger.critical('add_job(): Missing input {} for plugin {}'.format(inp['name'],plugin.name))
            elif inp['name'] not in inputs.keys():
                continue
            self._logger.debug('add_job(): {}={}'.format(inp['name'],inputs[inp['name']]))
            payload['parameters'][inp['name']] = inputs[inp['name']]
            
            # If input has {{ }}, then it has a dependency
            if isinstance(inputs[inp['name']],str):
                dependency = re.match(dependency_pattern,inputs[inp['name']])
                if dependency != None:
                    self._logger.info('add_job(): adding dependency {}'.format(dependency.groups()[0]))
                    payload['dependencies'].append(dependency.groups()[0])
        
        job = WippJob(data=payload)
        
        return job
        
class WippImageCollection(WippData):
    """ Class to handle WIPP Image Collections

    Attributes:
        name: the name given to the WIPP Image Collection
        id: a unique hash assigned to the WIPP Image Collection
        json: the raw json returned by the WIPP Workflow backend query
    
    Class Methods:
        get_all(): Returns a dictionary of all image collections, {image collection hash: WippImageCollection object}
        get(icid): Returns image collection with hash equal to icid
        get_by_name(ic_name): Returns the first result of a search of image collections matching ic_name
    
    Object Methods:
        create(): Create the workflow in WIPP.
        images(): Return a list of dictionaries containing information on every image in the collection
    """
    _entry_point = 'imagesCollections'
    _data_type_name = 'Image Collection'
    _images = []
    _logger = logging.getLogger('wipp.Data.WippImageCollection')
    
    def delete(self):
        """ Throw an error only if image collection is locked"""
        self._logger.info('delete(): deleting image collection - {}'.format(self.name))
        if self.locked:
            self._logger.critical('delete(): Cannot delete locked image collection.')
            raise PermissionError('Cannot delete locked image collection.')
        else:
            super().delete()
    
    @classmethod
    def get_by_name(cls,ic_name):
        cls._logger.info('get_by_name(): getting image collection - {}'.format(ic_name))
        r = requests.get(cls.api_route + cls._entry_point + '/search/findByName',params={'name':ic_name})
        cls._logger.debug('get_by_name(): status_code={}'.format(r.status_code))
        if r.status_code==200:
            imageCollection = cls(json=r.json()['_embedded'][cls._entry_point][0])
        else:
            imageCollection = []
        return imageCollection
    
    def update(self):
        self._logger.info('update(): updating image collection - {}'.format(self.name))
        ic = WippImageCollection.get_by_name(self.name)
        for key,value in ic.json.items():
            setattr(self,key,value)

    def add_image(self,file_path):
        self._logger.info('add_image(): file_path={}'.format(file_path))
        if self.locked:
            self._logger.info('add_image(): cannot add image to locked collection')
            raise PermissionError('Cannot add images to locked collection.')
        if not isinstance(file_path,Path):
            file_path = Path(file_path)
        return WippImage(self.id,file_path)
    
    def lock(self):
        self._logger.info('lock(): locking imaging collection - {}'.format(self.name))
        r = requests.patch(self.api_route + self._entry_point + '/' + self.id,
                           headers={'Content-Type': 'application/json'},
                           data=json_lib.dumps({'locked': True}))
        self._logger.debug('lock(): status_code={}'.format(r.status_code))
        
    def images(self):
        self._logger.info('images(): getting all images for image collection - {}'.format(self.name))
        if len(self._images) > 0 and self.locked:
            return self._images
        page = 0
        numel = 1000
        images = []
        r = self._get(entrypoint=self._entry_point + '/' + self.id + '/images',
                      params={'page':page,'size':numel})
        if '_embedded' not in r.keys():
            return images
        
        images = r['_embedded']['images']
        
        for i in range(r['page']['totalPages']-1):
            page += 1
            r = requests._get(self.api_route + self._entry_point + '/' + self.id + '/images',
                              params={'page':page,'size':numel})
            images.extend(r['_embedded']['images'])
        self._images = images
        return images
    
class WippCsvCollection(WippData):
    """ Class to handle WIPP Csv Collections

    Attributes:
        name: the name given to the WIPP csv collection
        id: a unique hash assigned to the WIPP csv collection
        json: the raw json returned by the WIPP csv collection backend query
        
    Class Methods:
        all(): Returns a dictionary of all csv collections, {csv collection hash: WippCsvCollection object}
    
    Object Methods:
        delete(): Delete the csv collection from WIPP.
        create(): Create the csv collection in WIPP.
    """
    _entry_point = 'csvCollections'
    _data_type_name = 'CSV Collection'
    _logger = logging.getLogger('wipp.Data.WippCsvCollection')
    
class WippNotebook(WippData):
    """ Class to handle WIPP Notebook

    Attributes:
        name: the name given to the WIPP notebook
        id: a unique hash assigned to the WIPP notebook
        json: the raw json returned by the WIPP notebook backend query
        
    Class Methods:
        all(): Returns a dictionary of all notebooks, {notebook hash: WippNotebook object}
    
    Object Methods:
        delete(): Delete the notebook from WIPP.
        create(): Create the notebook in WIPP.
    """
    _entry_point = 'notebooks'
    _data_type_name = 'Notebook'
    _logger = logging.getLogger('wipp.Data.WippNotebook')

class WippStitchingVector(WippData):
    """ Class to handle WIPP Stitching Vectors

    Attributes:
        name: the name given to the WIPP stitching vector
        id: a unique hash assigned to the WIPP stitching vector
        json: the raw json returned by the WIPP stitching vector backend query
    
    Class methods:
        all(): Returns a dictionary of all stitching vectors, {stitching vector hash: WippStitchingVector object}
        
    Object Methods:
        delete(): Delete the stitching vector from WIPP.
        create(): Create the stitching vector in WIPP.
    """
    _entry_point = 'stitchingVectors'
    _data_type_name = 'Stitching Vector'
    _logger = logging.getLogger('wipp.Data.WippStitchingVector')

class WippPyramid(WippData):
    """ Class to handle WIPP Pyramid

    Attributes:
        name: the name given to the WIPP pyramid
        id: a unique hash assigned to the WIPP pyramid
        json: the raw json returned by the WIPP pyramid backend query
    
    Class Methods:
        all(): Returns a dictionary of all pyramids, {pyramid hash: WippPyramid object}
        
    Object Methods:
        delete(): Delete the pyramid from WIPP.
        create(): Create the pyramid in WIPP.
    """
    _entry_point = 'pyramids'
    _data_type_name = 'Image Pyramid'
    _logger = logging.getLogger('wipp.Data.WippPyramid')

class WippImage(object):
    """ Class to handle WIPP Images

    Unlike most other WIPP classes, the WippImage class acts very differently from the
    other data types. Part of this comes from images being a child of an image collection
    and therefore necessitates attachment to a WippImageCollection id.

    In general, the best way to instantiate this class is through an WippImageCollection
    object using either the images() method to get all images in a collection or the
    add_image() method to prepare an image to upload to an unlocked collection.

    Attributes:
        To be determined
    
    Class Methods:
        To be determined
    
    Object Methods:
        delete(): Delete the image from an unlocked collection in WIPP.
        send(): Send the image to WIPP.
    """
    _entry_point = 'imagesCollections/{}/images'
    _data_type_name = 'Image'
    _flowChunkSize = 1048576
    _logger = logging.getLogger('wipp.Data.WippImage')

    def __init__(self,ic_id,file_path):
        self._entry_point = WippData.api_route + self._entry_point.format(ic_id)
        self.file_path = Path(file_path)
        if not self.file_path.is_file():
            self._logger.critical('__init__(): could not find file - {}'.format(str(self.file_path.absolute())))
            raise FileNotFoundError('Could not find file: {}'.format(str(self.file_path.absolute())))

        with open(self.file_path,'rb') as in_file:
            in_file.seek(0,2)
            self._flowTotalSize = in_file.tell()
            self._flowTotalChunks = self._flowTotalSize//self._flowChunkSize
        
        self._flowFilename = self.file_path.name

        self.params = {'flowChunkNumber': 1,
                       'flowChunkSize': self._flowChunkSize,
                       'flowCurrentChunkSize': self._flowChunkSize,
                       'flowTotalSize': self._flowTotalSize,
                       'flowIdentifier': str(self._flowTotalSize) + '-' + self.file_path.name.replace('.',''),
                       'flowFilename': self.file_path.name,
                       'flowRelativePath': self.file_path.name,
                       'flowTotalChunks': self._flowTotalChunks}
        
        for key,val in self.params.items():
            self._logger.debug('__init__(): {}={}'.format(key,val))
        
    def get_name(self):
        self._logger.info('get_name(): name={}'.format(self.file_path.name))
        return self.file_path.name
        
    def set_name(self,name):
        self._logger.info('set_name(): name={}'.format(name))
        suffix = ''.join(self.file_path.suffixes)
        name = name.split('.')[0] + suffix
        self.params['flowIdentifier'] = str(self._flowTotalSize) + '-' + name.replace('.','')
        self._logger.debug('set_name(): flowIdentifier={}'.format(self.params['flowIdentifier']))
        self.params['flowFilename'] = name
        self._logger.debug('set_name(): flowFilename={}'.format(self.params['flowFilename']))
        self.params['flowRelativePath'] = name
        self._logger.debug('set_name(): flowRelativePath={}'.format(self.params['flowRelativePath']))
        
    def send(self):
        self._logger.info('send(): file={}'.format(self.file_path))
        with open(self.file_path,'rb') as in_file:
            for chunk in range(1,self._flowTotalChunks):
                self._logger.debug('send(): sending chunk {} of {} for file {}'.format(chunk,self._flowTotalChunks,self.file_path))
                self.params['flowChunkNumber'] = chunk
                for retry in range(0,10):
                    try:
                        r = requests.post(self._entry_point,
                                        params=self.params,
                                        headers={'Content-Type': 'image/tiff'},
                                        data=in_file.read(1048576))
                        break
                    except:
                        if retry==9:
                            print('{}: Reached max tries.'.format(self.params['flowFilename']))
                            raise
                        print('{}: There was an upload error, will retry in 3 seconds (try {})'.format(self.params['flowFilename'],retry+1))
                        in_file.seek(-1048576,1)
                        time.sleep(3)
            self.params['flowChunkNumber'] = self._flowTotalChunks
            self.params['flowCurrentChunkSize'] = self._flowTotalSize-in_file.tell()
            self._logger.debug('send(): sending chunk {} of {} for file {}'.format(self._flowTotalChunks,self._flowTotalChunks,self.file_path))
            r = requests.post(self._entry_point,
                              params=self.params,
                              headers={'Content-Type': 'image/tiff'},
                              data=in_file.read(self._flowTotalSize-in_file.tell()))

class WippTensorflowModel(WippData):
    """ Class to handle WIPP Tensorflow Models

    Attributes:
        name: the name given to the WIPP tensorflow model
        id: a unique hash assigned to the WIPP tensorflow model
        json: the raw json returned by the WIPP tensorflow model backend query
    
    Class Methods:
        all(): Returns a dictionary of all models, {tensorflow model hash: WippTensorflowModel object}
    
    Object Methods:
        delete(): Delete the tensorflow model from WIPP.
        create(): Create the tensorflow model in WIPP.
    """
    _entry_point = 'tesorflowModels'
    _data_type_name = 'Tensorflow Models'
    _logger = logging.getLogger('wipp.Data.WippTensorflowModel')
        
class WippPlugin(WippData):
    """ Class to handle WIPP Plugins

    Attributes:
        name: the name given to the WIPP plugin
        id: a unique hash assigned to the WIPP plugin
        json: the raw json returned by the WIPP plugin backend query
        version: the plugin version
        inputs: a dictionary containing plugin inputs and settings
        outputs: a dictionary containing plugin output types and settings
        ui: a dictionary containing ui settings
    
    Class Methods:
        all(): Returns a dictionary of all plugins, {plugin hash: WippPlugin object}
        
    Object Methods:
        delete(): Delete the plugin from WIPP.
        create(): Create the plugin in WIPP.
    """
    _entry_point = 'plugins'
    _data_type_name = 'Plugin'
    _logger = logging.getLogger('wipp.Data.WippPlugin')

    # Get the newest plugin that matches a plugin name
    @classmethod
    def get_by_name(cls,name,version=None):
        cls._logger.info('get_by_name(): name={}, version={}'.format(name,version))
        all_plugins = cls.all().values()
        matching_plugins = [p for p in all_plugins if p.name==name]
        
        # If there are no matching plugins, throw an error
        if len(matching_plugins)==0:
            raise ValueError('No plugins match the supplied name: {}'.format(name))
        
        # If no version provided, get the latest version of the plugin
        if version == None:
            # If only one plugin matches, return that
            if len(matching_plugins)==1:
                return matching_plugins[0]
            
            version = [0,0,0] # major, minor, patch
            
            for p in reversed(matching_plugins):
                c_ver = re.match(r"([0-9]+).([0-9]+).([0-9]+)-?(.*)?",p.version)
                for i in range(3):
                    v = version[i]
                    c = c_ver.groups()[i]
                    if int(c) > v:
                        plugin = p
                        version = [int(v) for v in p.version.split('.')]
                        break
                    elif int(c) < v:
                        break
        # Return specified version of plugin
        else:
            for p in reversed(matching_plugins):
                if p.version==version:
                    return p
            # If the specified version could not be found, throw an error
            raise ValueError('Version {} of plugin {} was not found in WIPP. Try installing it.'.format(version,name))
            
        return plugin
        
    def __repr__(self):
        return f'{self.name} (id: {self.id}, version: {self.version})'
    
    @classmethod
    def all(cls):
        return super().all(entry_point='plugins/')
    
    @classmethod
    def install(cls,json):
        cls._logger.info('install(): installing plugin...')
        cls._logger.debug('install(): json={}'.format(json))
        p = cls(data=json)
    
        

In [ ]:

 """ 
UI Components and Logic for the Neuroglancer Notebook

Last Updated: May 5, 2020

"""

import ipywidgets as widgets
from IPython.display import display, clear_output
import IPython.display
import urllib
import copy
import pandas as pd
import numpy as np
import os





# API url to the pyramid info file
WIPP_API_URL=''

# link to local Neuroglancer deployment
NEUROGLANCER_URL= ''

# neuroglancer source link to load precomputed pyramids
PRECOMPUTED_URL='precomputed://' + WIPP_API_URL




pyramids = {}
value=''
WIPP_TYPES = {'plugin':'#66c2a5',
              'collection':'#fc8d62',
              'stitchingVector':'#8da0cb',
              'csvCollection':'#e78ac3',
              'pyramid':'#a6d854',
              'tensorflowModel':'#ffd92f'}

neuroglancer_images = []
shader_generator = """
void main() {{
    float brightness = {0:.6f};
    float contrast = {1:.4f};
    float red = {2:.3f};
    float green = {3:.3f};
    float blue = {4:.2f};
    emitRGB(vec3(
                 (toNormalized(getDataValue())+brightness) * exp(contrast) * red,
                 (toNormalized(getDataValue())+brightness) * exp(contrast) * green,
                 (toNormalized(getDataValue())+brightness) * exp(contrast) * blue
                 )
            );
}}
"""
empty_layer = {"source": "precomputed://",
               "type": "image",
               "blend": "additive",
               "shaderControls": {},
               "name": "No Name",
               "visible":True
               }

# Initialize neuroglancer_json
# changes in this are made to update the URL

neuroglancer_json = {
  "dimensions": {
    "x": [
      3.25e-7,
      "m"
    ],
    "y": [
      3.25e-7,
      "m"
    ],
    "z": [
      3.25e-7,
      "m"
    ]
  },
  "position": [
    21500.00,
    14600.00,
    0.5
  ],
  "crossSectionScale": 18,
  "projectionScale": 65536,
  "layers": [   
  ],
  "layout": "xy"
}


def get_selection_id(selection):
    selection_id = re.match(r".* \(id: ([0-9A-Za-z]+).*\)",selection)
    return selection_id.groups()[0]

def update_image_url(ind):
    global neuroglancer_json
    neuroglancer_json['layers'][ind]['source'] = PRECOMPUTED_URL.format(neuroglancer_images[ind]['pyramid'],neuroglancer_images[ind]['image'])    
    neuroglancer_json['layers'][ind]['shader'] = shader_generator.format(neuroglancer_images[ind]['brightness'],
                                                                         neuroglancer_images[ind]['contrast'],
                                                                         neuroglancer_images[ind]['red'],
                                                                         neuroglancer_images[ind]['green'],
                                                                         neuroglancer_images[ind]['blue'])
    neuroglancer_json['layers'][ind]['name'] = neuroglancer_images[ind]['image']


    
def update_url():
    global iframe 
    global value
    value = NEUROGLANCER_URL + urllib.parse.quote(json_lib.dumps(neuroglancer_json))   
    iframe.value='<iframe src="' + value + '" style="width:100%;height:98vh;"></iframe>' 
    

def bc_observer(im_index,key):
    def call_back(*args):
        global neuroglancer_images
        global neuroglancer_json
        neuroglancer_images[im_index][key] = args[0]['new']
        neuroglancer_json['layers'][im_index]['shader'] = shader_generator.format(neuroglancer_images[im_index]['brightness'],
                                                                         neuroglancer_images[im_index]['contrast'],
                                                                         neuroglancer_images[im_index]['red'],
                                                                         neuroglancer_images[im_index]['green'],
                                                                         neuroglancer_images[im_index]['blue'])
        update_url()
    return call_back

def zoom_observer():
    def call_back(*args):
        global neuroglancer_json
        zoom_scale = (1001-args[0]['new'])/10
        neuroglancer_json['crossSectionScale']=zoom_scale
        update_url()
    return call_back

def x_observer(ind):
    def call_back(*args):
        global neuroglancer_json       
        if ind==0:
            neuroglancer_json['position'][ind]=args[0]['new']
        if ind==1:
            neuroglancer_json['position'][ind]=65000-args[0]['new']         
        update_url()
    return call_back
    

def parse_color(im_index):
    def call_back(*args):
        global neuroglancer_images
        neuroglancer_images[im_index]['red'] = int(args[0]['new'][1:3],16) / 255
        neuroglancer_images[im_index]['green'] = int(args[0]['new'][3:5],16) / 255
        neuroglancer_images[im_index]['blue'] = int(args[0]['new'][5:7],16) / 255      
        update_image_url(im_index)
        update_url() 
    return call_back

def channel_observer():
    def call_back(*args):
        global neuroglancer_json     
        neuroglancer_json['layers'][0]['visible']=False            
        update_url()
    return call_back

def layout_observer():
    def call_back(*args):
        global neuroglancer_json        
        neuroglancer_json['layout']=args[0]['new']
        update_url()
    return call_back       
        
    
def checkbox_observer(ind):
    def call_back(*args):
        global neuroglancer_json
        value=args[0]['new']
        if value==True:
            neuroglancer_json['layers'][ind]['visible']=True
        elif value==False:
            neuroglancer_json['layers'][ind]['visible']=False
        update_url()
    return call_back          
    
    
def img_observer(b_widgets,im_index):
    def call_back(*args):
        global neuroglancer_images        
        global channels

        for b_widget in b_widgets:
            b_widget.disabled = False
        neuroglancer_images[im_index]['image'] = args[0]['new']      
       
        update_image_url(im_index)
        update_url()
        new_widget=widgets.Checkbox( value=True,    description= args[0]['new'],disabled=False)
        channels.children=tuple(list(channels.children)+[new_widget])
        channels.children[-1].observe(checkbox_observer(len(channels.children)-1))
    return call_back

def pyr_observer(im_widget,im_index):
    def call_back(*args):
        global neuroglancer_images
        
        selection = get_selection_id(args[0]['new'])
        
        if selection==None :
            return
        pyr = WippPyramid.get_by_id(selection)        
        local_dir= "/opt/shared/wipp/pyramids/{}".format(selection)
        image_names=os.listdir(local_dir)
        im_widget.disabled=False           
        im_widget.options=[filename for filename in image_names]
        neuroglancer_images[im_index]['pyramid'] = pyr.id                                     
                 
    return call_back    


def CopyURL(*args):
    global display_url
    global value    
    display_url.value=value 

# Filter out neuroglancer type pyramids
precomputed_pyramids=[str(ic) for ic in WippPyramid.all().values() if 'pyramidType' in WippJob.get_by_id(WippPyramid.get_by_id(get_selection_id(str(ic))).job).parameters.keys() ]
precomputed_pyramids=[ic for ic in precomputed_pyramids if WippJob.get_by_id(WippPyramid.get_by_id(get_selection_id(str(ic))).job).parameters['pyramidType']=='Neuroglancer']

def add_pyramid(*args):
    global pyramids    
    pyr = widgets.Combobox(placeholder='Click on the box or start typing!',
                           options=precomputed_pyramids,
                           description='Pyramid:',
                           ensure_option=True,
                           disabled=False,
                           layout=widgets.Layout(width='95%'))
    
    imc = widgets.Combobox(placeholder='Select a pyramid first!',
                           options=[],
                           description='Image:',
                           ensure_option=True,
                           disabled=False,
                           layout=widgets.Layout(width='95%'))
    
    brightness = widgets.FloatSlider(value=0,
                                     min=-1,
                                     max=1,
                                     step=0.01,
                                     description="Brightness:",
                                     continuous_update=False,
                                     orientation='horizontal',
                                     readout=False,
                                     layout=widgets.Layout(width='68%'),
                                     disabled=True)
    
    brightness_ticker = widgets.FloatText(value=0,
                                          step=0.01,
                                          continuous_update=False,
                                          orientation='horizontal',
                                          layout=widgets.Layout(width='28%'),
                                          disabled=True)
    
    contrast = widgets.FloatSlider(value=0,
                                   min=-3,
                                   max=3,
                                   step=0.01,
                                   description="Contrast:",
                                   continuous_update=False,
                                   orientation='horizontal',
                                   readout=False,
                                   layout=widgets.Layout(width='68%'),
                                   disabled=True)
    
    contrast_ticker = widgets.FloatText(value=0,
                                        min=-3,
                                        max=3,
                                        step=0.01,
                                        continuous_update=True,
                                        orientation='horizontal',
                                        layout=widgets.Layout(width='28%'),
                                        disabled=True)   

    
    color = widgets.ColorPicker(concise=False,
                                description='Color:',
                                value='#ffffff',
                                layout=widgets.Layout(width='68%'),
                                disabled=True)
    widgets.jslink((brightness_ticker, 'value'), (brightness, 'value'))
    widgets.jslink((contrast_ticker, 'value'), (contrast, 'value'))
    
    
    neuroglancer_images.append({'pyramid': None,
                                'image':None,
                                'brightness':0,
                                'contrast':0,
                                'red':1,
                                'green':1,
                                'blue':1})
    
    neuroglancer_json['layers'].append(copy.deepcopy(empty_layer))
    pyr.observe(pyr_observer(imc,len(pyramids.children)),'value')
    imc.observe(img_observer([brightness,brightness_ticker,contrast,contrast_ticker,color],len(pyramids.children)),'value')
    brightness.observe(bc_observer(len(pyramids.children),'brightness'),'value')
    contrast.observe(bc_observer(len(pyramids.children),'contrast'),'value')
    
    color.observe(parse_color(len(pyramids.children)),'value')
    pyramids.children = pyramids.children + (widgets.VBox([pyr,
                                                           imc,
                                                           widgets.HBox([brightness,brightness_ticker]),
                                                           widgets.HBox([contrast,contrast_ticker]),                                                           
                                                           color]),)
    pyramids.set_title(len(pyramids.children)-1,'Select pyramid...')
    
def wipp_type_observer(j_input,j_workflow):
    def call_back(*args):
        selection = get_selection_id(args[0]['new'])
        if selection==None:
            return
        j_input['value'] = selection
        if display_workflow:
            draw_workflow(j_workflow)
    return call_back



# Add Zoom Slider and ticker
zoom_slider = widgets.FloatSlider(value=720,
                               min=1,
                               max=1000,
                               step=1,
                               description="Zoom:",
                               continuous_update=False,
                               orientation='horizontal',
                               readout=False,
                               layout=widgets.Layout(width='72%'),
                               disabled=False)

zoom_ticker = widgets.FloatText(value=0,
                                    min=0.00,
                                    max=100,
                                    step=1,
                                    continuous_update=True,
                                    orientation='horizontal',
                                    layout=widgets.Layout(width='25%'),
                                    disabled=False)

#link zoom slider and ticker
widgets.jslink((zoom_slider, 'value'), (zoom_ticker, 'value'))
# set callback function for zoom
zoom_slider.observe(zoom_observer(),names='value')



# Add x-location slider and ticker
x_slider = widgets.FloatSlider(value=21500,
                               min=0.00,
                               max=65000.00,
                               step=100,
                               description="x location:",
                               continuous_update=False,
                               orientation='horizontal',
                               readout=False,
                               layout=widgets.Layout(width='72%'),
                               disabled=False)

x_ticker = widgets.FloatText(value=21500,
                                    min=0.00,
                                    max=65000.00,
                                    step=100,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    layout=widgets.Layout(width='25%'),
                                    disabled=False)

# link x slider and ticker
widgets.jslink((x_slider, 'value'), (x_ticker, 'value'))
# set callback function for x slider
x_slider.observe(x_observer(0),'value')



# Add y-location slider and ticker
y_slider = widgets.FloatSlider(value=65000-14600,
                               min=0,
                               max=65000,
                               step=100,
                               description="y location:",
                               continuous_update=False,
                               orientation='vertical',
                               readout=False,
                               layout=widgets.Layout(width='72%'),
                               disabled=False)

y_ticker = widgets.FloatText(value=14600,
                                    min=0.00,
                                    max=65000.00,
                                    step=100,
                                    continuous_update=True,
                                    orientation='horizontal',
                                    layout=widgets.Layout(width='25%'),
                                    disabled=False)

# link y slider and ticker
widgets.jslink((y_slider, 'value'), (y_ticker, 'value'))
# set call back function for y slider
y_slider.observe(x_observer(1),'value')


# add widget to change layout
layout=widgets.ToggleButtons(options=["xy","yz","xz","4panel"],                              
                             disabled=False,
                             description='Layout:         '  ,                   
                    
                             layout=widgets.Layout(width='50%'))
layout.observe(layout_observer(),'value')


# Set up display of widgets

# navigation panel
navigation_panel=widgets.VBox([widgets.HBox([zoom_slider,zoom_ticker]),widgets.HBox([x_slider,x_ticker]),widgets.HBox([y_slider,y_ticker])])
navigation_accordion=widgets.Accordion(children=[navigation_panel])
navigation_accordion.set_title(0,'Navigation Panel')

# selection panel
channels=widgets.VBox(children=[])
channel_list= widgets.HBox([widgets.Label(value="Channels: "), channels])
layout_list= widgets.HBox([ layout])
out1=widgets.VBox([channel_list,layout_list])
selection_panel=widgets.VBox([out1])
selection_accordion=widgets.Accordion(children=[selection_panel])
selection_accordion.set_title(0,'Selection Panel')

# add widget to copy and display current url
copy_url = widgets.Button(description='Copy Url')
display_url=widgets.Textarea( value= 'This text is enclosed in a box',disabled= True,layout=widgets.Layout(width='70%'))
copy_url.on_click(CopyURL)
url=widgets.HBox([copy_url,display_url])

# UI components
add_channel = widgets.Button(description='Add Channel')
pyramids = widgets.Accordion(children=[],
                             description='Job inputs:')


add_pyramid()
add_channel.on_click(add_pyramid)

value='<iframe src="{}" style="width:100%;height:98vh;"></iframe>'.format(NEUROGLANCER_URL)
iframe = widgets.HTML(value=value,layout=widgets.Layout(width='100%',height='100%'))
update_url()

display(widgets.HBox([widgets.VBox([pyramids,add_channel,url,navigation_accordion,selection_accordion]),iframe]))


